# Classifying six-figures in malpractice payments

## Imports and taking peek at the data

In [ ]:
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
from scipy import stats
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib as mpl
warnings.filterwarnings('ignore')
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score, cross_validate, ShuffleSplit, train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from scipy.stats import chi2_contingency
from sklearn.linear_model import LogisticRegression

In [ ]:
df = pd.read_csv('NPDB2401_Modern_Malpractice_Clean.csv')
df

In [ ]:
df.info()

### Let's see the distrubtion of the payment column 

In [ ]:
sns.histplot(data=df, x='PAYMENT')

### Let's scale that distrubtion

In [ ]:
sns.histplot(data=df, x='PAYMENT', log_scale=True)

### Should we classify by the digits of the payment?

In [ ]:
log_ref = {}
for i in range(2,9):
    log_ref[i] = len(df[(df['PAYMENT'] > 10**(i-1)) & (df['PAYMENT'] <= 10**(i))])
log_ref 

### We should not:
   - Massive class imbalace, most payment are 6 figures 
   - The smaller figuares will be problematic noise
### What will do instead:
   - Classify payments as under, over, or extacly 6 figures using orinal encoding and model for that

In [ ]:
df['PAYMENT'] = df['PAYMENT'].map(lambda x: math.ceil(math.log(x, 10)) ).astype(int)
df['PAYMENT'] = df['PAYMENT'].map(lambda x: 2 if x > 6 else (0 if x < 6 else 1))

In [ ]:
log_ref = {}
for i in range(df['PAYMENT'].min(),df['PAYMENT'].max()+1):
    log_ref[i] = len(df[df['PAYMENT'] == i])
log_ref 

### Better, Next Let's look at some numerical correlations to see how each column could affect our models

In [ ]:
num_col = ['PAYMENT','GRAD','PRACTAGE', 'NPMALRPT', 'NPLICRPT', 'NPCLPRPT' , 'NPPSMRPT' , 'NPDEARPT', 'NPEXCRPT', 'NPGARPT' ,'NPCTMRPT', 'MALTIME', 'NUMBPRSN', 'PTAGE']
num_df = df[num_col]

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(num_df.corr(), cmap="YlGnBu", annot=True, fmt=".2f")
plt.show()

### The strongest correlations by magnitudes are:
   - Grad and Practage
   - NPExcrpt and NPMalrpt
### Because of that we will remove:
   - Practage and NPExcrpt since Grad and NPMalrpt are more correlated to payments
   - because of the high correlation Practage and NPExcrpt are redundant 

In [ ]:
df.drop(df[['PRACTAGE', 'NPEXCRPT']], axis=1, inplace=True)
num_col = list(set(num_df.columns) - set(['PRACTAGE', 'NPEXCRPT','PAYMENT']))
num_df = df[num_col]

### What about the categorical correlations?

### Before that we have clean out the irregular categories from our categorical columns with a lot of categories

In [ ]:
cat_cols = list(set(df.columns) - set(num_df.columns))

In [ ]:
col_ref = {}
for col in cat_cols:
    col_ref[col]  = len(df[col].unique())
    df[col] = df[col].astype(str)
col_ref

### 15 or more catagories counts as categorical column with a lot of categories

In [ ]:
big_cols = [col for col in col_ref.keys() if col_ref[col] > 15]
big_cols

### If the catagory appears in less than 1% of the data it will be replaced by a V for Various 

In [ ]:
for col in big_cols:
    ref = dict(df[col].value_counts())
    df[col] = df[col].map(lambda x: 'V' if ref[x] < 10**-2*len(df) else x)

In [ ]:
col_ref = {}
for col in cat_cols:
    col_ref[col]  = len(df[col].unique())
col_ref

### Better, now for some more quick cleaning

In [ ]:
for col in cat_cols:
    if len(df[col].unique()) == 2: 
        print(f'{col} : {df[col].unique()}')

In [ ]:
df['PAYNUMBR'] = df['PAYNUMBR'].map(lambda x: '0' if x == 'S' or x == '0' else '1')
for col in cat_cols:
    if len(df[col].unique()) == 2: 
        print(f'{col} : {df[col].unique()}')

### For categorical correlations we have to use cramers_v,  an effect size measurement for the chi-square test of independence,  as our metric

In [ ]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))


### Let's get those correlations in a heatmap

In [ ]:
cat_cor_cols = cat_cols
corr_matrix = pd.DataFrame(index=cat_cor_cols, columns=cat_cor_cols)
for col1 in cat_cor_cols:
    for col2 in cat_cor_cols:
        if col1 == col2:
            corr_matrix.loc[col1, col2] = 1.0
        else:
            corr_matrix.loc[col1, col2] = cramers_v(df[col1], df[col2])
    

In [ ]:
corr_matrix = corr_matrix.astype(float)

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=0, vmax=1, fmt='.2f')
plt.title('Correlation Heatmap for Categorical Variables')
plt.show()

### We can see:
   - STATEFUND strongly correlates with a handful of the cataories and should be droped
   - ISINSURE strongly correlates with a couple of the cataories and should be droped
   - LICNSTAT correlates well with WORKSTATE and we have stat if both match, and suits are most likly filed in the same state WORKSTATE, LICNSTAT should be droped

In [ ]:
df.drop(df[['STATEFUND', 'LICNSTAT', 'ISINSURE']], axis=1, inplace=True)
cat_cols = list(set(cat_cols) - set(['STATEFUND', 'LICNSTAT', 'ISINSURE', 'PAYMENT']))

## Modeling Time

### First training and test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['PAYMENT'], axis=1) , df['PAYMENT'], test_size=.3, random_state = 31)

### Here an example of how the cataorical columns are encoded, and the columns for both train and test

In [ ]:
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
dummies = ohe.fit_transform(X_train[cat_cols])
X_train_onehot = pd.DataFrame(dummies, columns=ohe.get_feature_names_out(), index=X_train.index)
X_train_Enc = pd.concat([X_train[num_col], X_train_onehot], axis=1)
scaler = StandardScaler()
X_train_Enc_ss = scaler.fit_transform(X_train_Enc)
len(X_train) == len(X_train_Enc_ss)

In [ ]:
dummies = ohe.transform(X_test[cat_cols])
X_test_onehot = pd.DataFrame(dummies, columns=ohe.get_feature_names_out(), index=X_test.index)
X_test_Enc = pd.concat([X_test[num_col], X_test_onehot], axis=1)
X_test_Enc_ss = scaler.transform(X_test_Enc)
len(X_test) == len(X_test_Enc_ss)

### Let's try a simple Decision Tree Classifier

In [ ]:
model = DecisionTreeClassifier(criterion = 'gini',  max_depth = 4, random_state=42)
model.fit(X_train_Enc_ss, y_train)
accuracy_train = model.score(X_train_Enc_ss, y_train)
accuracy_train

### What were the important features, and how important are they

In [ ]:
imp_ref = {X_train_Enc.columns[i]: model.feature_importances_[i] for i in range(len(X_train_Enc.columns)) if model.feature_importances_[i] != 0}
plt.figure(figsize=(10, 6))
plt.bar(imp_ref.keys(), imp_ref.values())
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances in Decision Tree')
plt.xticks(rotation=45)
plt.show()

### Let's take a peek into the tree

In [ ]:
f, ax = plt.subplots(figsize=(50,30))
plot_tree(model, ax=ax, fontsize=10);

### Is there a risk for variance?

### Let's make our own cross_validation method to pervent data leakage and keep our encoding and scaling

In [ ]:
def cross_validation(X_train, y_train, model, num_split = 10):
        
    score_train_list = []
    score_val_list = []
    
    for train_index, valid_index in KFold(n_splits = num_split).split(X_train):
        
        # train and validation splitting 
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[valid_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[valid_index]
        
        ohe = OneHotEncoder(sparse_output=False)
        dummies = ohe.fit_transform(X_train_fold[cat_cols])
        X_train_onehot = pd.DataFrame(dummies, columns=ohe.get_feature_names_out(), index=X_train_fold.index)
        X_train_Enc = pd.concat([X_train_fold[num_col], X_train_onehot], axis=1)
        
        dummies = ohe.transform(X_val_fold[cat_cols])
        X_val_onehot = pd.DataFrame(dummies, columns=ohe.get_feature_names_out(), index=X_val_fold.index)
        X_val_Enc = pd.concat([X_val_fold[num_col], X_val_onehot], axis=1)

        #create/fit the Standard scaler on the train fold
        scaler = StandardScaler()
        X_tf_sc = scaler.fit_transform(X_train_Enc)
        # transform validation fold
        X_vld_sc = scaler.transform(X_val_Enc)

        # Score it
        model.fit(X_tf_sc, y_train_fold)
        
        # now how did we do?
        accuracy_train = model.score(X_tf_sc, y_train_fold)
        accuracy_val = accuracy_score(y_val_fold, model.predict(X_vld_sc))
        score_val_list.append(accuracy_val)
        score_train_list.append(accuracy_train)
    
    return {'train': np.mean(score_train_list), 'validation': np.mean(score_val_list)}

In [ ]:
cross_validation(X_train, y_train, model)

### There does not seem to be. 

### Let's try uping the max_depth

In [ ]:
g_model = DecisionTreeClassifier(criterion = 'gini',  max_depth = 7, random_state=42)
g_model.fit(X_train_Enc_ss, y_train)
accuracy_train = g_model.score(X_train_Enc_ss, y_train)
accuracy_train

In [ ]:
cross_validation(X_train, y_train, g_model)

### A seeming minmal risk of variance
### Important features?

In [ ]:
imp_ref = {X_train_Enc.columns[i]: g_model.feature_importances_[i] for i in range(len(X_train_Enc.columns)) if g_model.feature_importances_[i] > .01}
plt.figure(figsize=(10, 6))
plt.bar(imp_ref.keys(), imp_ref.values())
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances in Decision Tree (gini)')
plt.xticks(rotation=45)
plt.show()

### Same model but with a entropy criterion

In [ ]:
e_model = DecisionTreeClassifier(criterion = 'entropy',  max_depth = 7, random_state=42)
e_model.fit(X_train_Enc_ss, y_train)
accuracy_train = e_model.score(X_train_Enc_ss, y_train)
accuracy_train

In [ ]:
cross_validation(X_train, y_train, e_model, num_split = 10)

### A seeming minmal risk of variance
### Important features?

In [ ]:
imp_ref = {X_train_Enc.columns[i]: e_model.feature_importances_[i] for i in range(len(X_train_Enc.columns)) if e_model.feature_importances_[i] > .01}
plt.figure(figsize=(10, 6))
plt.bar(imp_ref.keys(), imp_ref.values())
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances in Decision Tree (entropy)')
plt.xticks(rotation=45)
plt.show()

### Let's see the final results for the entropy criterion model

In [ ]:
accuracy_score(y_test, e_model.predict(X_test_Enc_ss))

### In more detail

In [ ]:
print(classification_report(y_test, e_model.predict(X_test_Enc_ss)))

### In visual detail

In [ ]:
cfmat = confusion_matrix(y_test, e_model.predict(X_test_Enc_ss))

ConfusionMatrixDisplay(cfmat,display_labels=['Under', 'Right', 'Over']).plot()

### Let's see the final results for the gini criterion model with max_depth 7

In [ ]:
accuracy_score(y_test, g_model.predict(X_test_Enc_ss))

### In more detail

In [ ]:
print(classification_report(y_test, g_model.predict(X_test_Enc_ss)))

### In visual detail

In [ ]:
cfmat = confusion_matrix(y_test, g_model.predict(X_test_Enc_ss))

ConfusionMatrixDisplay(cfmat,display_labels=['Under', 'Right', 'Over']).plot()

### Lastly, our oringal base tree model

In [ ]:
accuracy_score(y_test, model.predict(X_test_Enc_ss))

### In more detail

In [ ]:
print(classification_report(y_test, model.predict(X_test_Enc_ss)))

### In visual detail

In [ ]:
cfmat = confusion_matrix(y_test, model.predict(X_test_Enc_ss))

ConfusionMatrixDisplay(cfmat,display_labels=['Under', 'Right', 'Over']).plot()

## Decision Tree Classifier Summary:

### Best model is gini criterion model with max_depth 7 by accuracy

### We can tell what is important, but not how impactful it for the payment.

## Let's try Logistical Regression
### We can find out only what is important, but how impactful it for the payment.
### Starting with a basic model

In [ ]:
model_log_basic = LogisticRegression()
model_log_basic.fit(X_train_Enc_ss, y_train)
model_log_basic.score(X_train_Enc_ss, y_train)

In [ ]:
cross_validation(X_train, y_train, model_log_basic, num_split = 10)

### A seeming minmal risk of variance
### Important weights?

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 12))
cat = ['Under', 'Exactly', 'Over']
thersh = [.15, .03, .15]
for i in range(3):
    imp_ref = {X_train_Enc.columns[k]: model_log_basic.coef_[i][k] for k in range(len(X_train_Enc.columns)) if np.abs(model_log_basic.coef_[i][k]) >= thersh[i]}
    sns.barplot(y=imp_ref.keys(), x=imp_ref.values(), orient='h', ax=axes[i])
    axes[i].set_xlabel('Importance')
    axes[i].set_ylabel('Features')
    axes[i].set_title(f'Weights for {cat[i]} six-figures in Basic Logistic Regression Model')
plt.tight_layout()
plt.show()

### Let's try optimizing 
### What is the best C?

In [ ]:
def log_hyper_opt(X_train, y_train):
    C_list = [1e-4,1e-3, 1e-2, 1e-1, 1, 10, 100, 1e3]
    cv_scores = {}
    for c in C_list:
        logreg = LogisticRegression(C = c)
        cv_loop_results = cross_validation(X_train, y_train, logreg, num_split = 10)
        cv_scores[c] = cv_loop_results['validation']
    return cv_scores

### WARING this script is UBER SLOW

In [ ]:
log_hyper_opt(X_train, y_train)

### C = 10 is  the best 

In [ ]:
model_log_c10 = LogisticRegression(C=10)
model_log_c10.fit(X_train_Enc_ss, y_train)
model_log_c10.score(X_train_Enc_ss, y_train)

### A seeming minmal risk of variance
### Important weights?

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 12))
cat = ['Under', 'Exactly', 'Over']
thersh = [.15, .03, .15]
for i in range(3):
    imp_ref = {X_train_Enc.columns[k]: model_log_c10.coef_[i][k] for k in range(len(X_train_Enc.columns)) if np.abs(model_log_c10.coef_[i][k]) >= thersh[i]}
    sns.barplot(y=imp_ref.keys(), x=imp_ref.values(), orient='h', ax=axes[i])
    axes[i].set_xlabel('Importance')
    axes[i].set_ylabel('Features')
    axes[i].set_title(f'Weights for {cat[i]} six-figures in Basic Logistic Regression Model with Optimal Regularization')
plt.tight_layout()
plt.show()

### Is there a better solver?
### WARING this script is UBER SLOW

In [ ]:
solver_ref = {}.fromkeys(['newton-cg', 'sag', 'saga', 'lbfgs']) 
for s in solver_ref.keys():
    model_log_c10_slove = LogisticRegression(C=10, solver=s)
    model_log_c10_slove.fit(X_train_Enc_ss, y_train)
    solver_ref[s] = model_log_c10_slove.score(X_train_Enc_ss, y_train)
solver_ref

### Newton-cg is the best 

In [ ]:
model_log_c10_newton = LogisticRegression(C=10, solver='newton-cg')
model_log_c10_newton.fit(X_train_Enc_ss, y_train)
cross_validation(X_train, y_train, model_log_c10_newton, num_split = 10)

### A seeming minmal risk of variance
### Important weights?

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 12))
cat = ['Under', 'Exactly', 'Over']
thersh = [.15, .03, .15]
for i in range(3):
    imp_ref = {X_train_Enc.columns[k]: model_log_c10.coef_[i][k] for k in range(len(X_train_Enc.columns)) if np.abs(model_log_c10.coef_[i][k]) >= thersh[i]}
    sns.barplot(y=imp_ref.keys(), x=imp_ref.values(), orient='h', ax=axes[i])
    axes[i].set_xlabel('Importance')
    axes[i].set_ylabel('Features')
    axes[i].set_title(f'Weights for {cat[i]} six-figures in Basic Logistic Regression Model with Optimal Regularization')
plt.tight_layout()
plt.show()

## Let's see the final results for the Logistical Regression models

### Basic Model

In [ ]:
accuracy_score(y_test, model_log_basic.predict(X_test_Enc_ss))

### In more detail

In [ ]:
print(classification_report(y_test, model_log_basic.predict(X_test_Enc_ss)))

### In visual detail

In [ ]:
cfmat = confusion_matrix(y_test, model_log_basic.predict(X_test_Enc_ss))

ConfusionMatrixDisplay(cfmat,display_labels=['Under', 'Right', 'Over']).plot()

### Default Solver with C=10

In [ ]:
accuracy_score(y_test, model_log_c10.predict(X_test_Enc_ss))

### In more detail

In [ ]:
print(classification_report(y_test, model_log_c10.predict(X_test_Enc_ss)))

### In visual detail

In [ ]:
cfmat = confusion_matrix(y_test, model_log_c10.predict(X_test_Enc_ss))

ConfusionMatrixDisplay(cfmat,display_labels=['Under', 'Right', 'Over']).plot()

### Solver with newton-cg with C=10

In [ ]:
accuracy_score(y_test, model_log_c10_newton.predict(X_test_Enc_ss))

### In more detail

In [ ]:
print(classification_report(y_test, model_log_c10_newton.predict(X_test_Enc_ss)))

### In visual detail

In [ ]:
cfmat = confusion_matrix(y_test, model_log_c10.predict(X_test_Enc_ss))

ConfusionMatrixDisplay(cfmat,display_labels=['Under', 'Right', 'Over']).plot()

## Logistic Regression Summary:
   
### They all preforms similarly but C=10 and lbfgs solver preforms the best

### Important notes
   - The higher the outcome, the more severe the injury to the patient (1 to 9 only)
   - Where the practitioner is working matters

# Summary

### Best model is a C=10 and lbfgs solver logistical regression and it call t

## Recomendtions:
   - If you are a malpratice insurce company look into spending more in advertising in New York, and illinois. According to our model thoes states tend to a have higher than a six-figures malpratice settlements. You can pull funding from advertising in Puerto Rico and to advertising to dentist, since those payments tend to be smaller than six-figures according to the model. 

   - As lawyer, as you may expect, outcome is the most important factor it worth focusing on that when coming to a decision of giving a six-figure settlment or something higher or lower than that. The more server the higher the outcome (1-9).
   
## What is next
   - An EDA to fine tune the recomendtions.
   - Maybe try a regression models, but use the classifier methods since the numerical data is encoded,for example PRACTAGE, PTAGE does not give the age of the practitioner or patient respectively rather an age group they belong to. On top of that over half of the data is categorical making hard of tradional regression to work properly.